# Displaying MusicBrainz data in a timeline

One thing I thought about adding to the MusicBrainz web interface is a timeline on band pages, in order to show when new members arrived or left the band.

Before suggesting this change to the MusicBrainz project, we can try to display this kind of timeline in a Jupyter notebook and see how it would look like. I decided to test the [timesheet-advanced.js](https://github.com/ntucakovic/timesheet-advanced.js) library for this purpose.

If you can't see the timelines on this page, you can find a "static" copy of them on [github.io](https://loujine.github.io/musicbrainz-dataviz/).

Again I need first to initialize a connection to my local copy of the MusicBrainz database:

In [1]:
from pprint import pprint

import pandas
import sqlalchemy
# define DB
PGHOST = "192.168.11.3"
PGDATABASE = "musicbrainz_db"
PGUSER = "musicbrainz"
PGPASSWORD = "musicbrainz"

engine = sqlalchemy.create_engine(
   'postgresql+psycopg2://{PGUSER}:{PGPASSWORD}@{PGHOST}/{PGDATABASE}'.format(**locals()),
    isolation_level='READ UNCOMMITTED')

# helper function
def sql(query, **kwargs):
    return pandas.read_sql(query, engine, params=globals(), **kwargs)

Now I can extract the information for the band I want. The SQL query will look for:

* band name
* artists linked to this band through the "member of" relationship
* instrument/vocal role of this relationship

Let's start with some band you probably already know:

In [2]:
band_name = 'The Beatles'

The SQL query is a bit complicated because it uses a lot of different tables

In [3]:
df = sql("""
SELECT b.name AS band,
       m.name AS member,
       m.gid AS mbid,
       lat.name AS role,
       l.begin_date_year AS start,
       l.end_date_year AS end
FROM artist              AS b
JOIN l_artist_artist     AS laa ON laa.entity1 = b.id
JOIN artist              AS m   ON laa.entity0 = m.id
JOIN link                AS l   ON l.id = laa.link
JOIN link_attribute      AS la  ON la.link = l.id
JOIN link_attribute_type AS lat ON la.attribute_type = lat.id
JOIN link_type           AS lt  ON l.link_type = lt.id
WHERE lt.name = 'member of band'
  AND b.name = %(band_name)s
  AND lat.name != 'original';
""")
df

,band,member,mbid,role,start,end
0,The Beatles,George Harrison,42a8f507-8412-4611-854f-926571049fa0,lead vocals,1958.0,1970
1,The Beatles,George Harrison,42a8f507-8412-4611-854f-926571049fa0,guitar,1958.0,1970
2,The Beatles,Pete Best,0d4ab0f9-bbda-4ab1-ae2c-f772ffcfbea9,drums,1960.0,1962
3,The Beatles,Paul McCartney,ba550d0e-adac-4864-b88b-407cab5e76af,lead vocals,1957.0,1970
4,The Beatles,Paul McCartney,ba550d0e-adac-4864-b88b-407cab5e76af,bass guitar,1957.0,1970
5,The Beatles,Ringo Starr,300c4c73-33ac-4255-9d57-4e32627f5e13,drums,1962.0,1970
6,The Beatles,Stuart Sutcliffe,49a51491-650e-44b3-8085-2f07ac2986dd,bass guitar,1960.0,1962
7,The Beatles,John Lennon,4d5447d7-c61c-4120-ba1b-d7f471d385b9,lead vocals,NaN,1970
8,The Beatles,John Lennon,4d5447d7-c61c-4120-ba1b-d7f471d385b9,guitar,NaN,1970


Looks good. Apart from some details:

* John Lennon's role has no starting date, we can set it to 1957
* these missing values made PanDas believe year dates are floating point number, we should fix this

In [4]:
df['start'] = df['start'].fillna(1957).astype(int)
df['start'] = df['start'].astype(str)
df['end'] = df['end'].astype(str)
df['mbid'] = df['mbid'].astype(str)
df

,band,member,mbid,role,start,end
0,The Beatles,George Harrison,42a8f507-8412-4611-854f-926571049fa0,lead vocals,1958,1970
1,The Beatles,George Harrison,42a8f507-8412-4611-854f-926571049fa0,guitar,1958,1970
2,The Beatles,Pete Best,0d4ab0f9-bbda-4ab1-ae2c-f772ffcfbea9,drums,1960,1962
3,The Beatles,Paul McCartney,ba550d0e-adac-4864-b88b-407cab5e76af,lead vocals,1957,1970
4,The Beatles,Paul McCartney,ba550d0e-adac-4864-b88b-407cab5e76af,bass guitar,1957,1970
5,The Beatles,Ringo Starr,300c4c73-33ac-4255-9d57-4e32627f5e13,drums,1962,1970
6,The Beatles,Stuart Sutcliffe,49a51491-650e-44b3-8085-2f07ac2986dd,bass guitar,1960,1962
7,The Beatles,John Lennon,4d5447d7-c61c-4120-ba1b-d7f471d385b9,lead vocals,1957,1970
8,The Beatles,John Lennon,4d5447d7-c61c-4120-ba1b-d7f471d385b9,guitar,1957,1970


To display the timeline inside this notebook we need to lead the JS/CSS source of the timesheet-advanced package:

In [5]:
from IPython.display import HTML
HTML("""
<link rel="stylesheet" type="text/css" href="./timesheet/timesheet.min.css" />
<script type="text/javascript" src="./timesheet/timesheet-advanced.min.js"></script>
""")

The timesheet-advanced package requires the input data for the timeline to be inserted slightly differently from what we have in our dataframe df. We need a 'label' field (we'll choose the band member name + instrument) and we need a 'type' which is a color. We choose colors to represent all possible roles (vocals, guitar, drums....)

In [6]:
df['label'] = df['member'] + ' (' + df['role'] + ')'
df

,band,member,mbid,role,start,end,label
0,The Beatles,George Harrison,42a8f507-8412-4611-854f-926571049fa0,lead vocals,1958,1970,George Harrison (lead vocals)
1,The Beatles,George Harrison,42a8f507-8412-4611-854f-926571049fa0,guitar,1958,1970,George Harrison (guitar)
2,The Beatles,Pete Best,0d4ab0f9-bbda-4ab1-ae2c-f772ffcfbea9,drums,1960,1962,Pete Best (drums)
3,The Beatles,Paul McCartney,ba550d0e-adac-4864-b88b-407cab5e76af,lead vocals,1957,1970,Paul McCartney (lead vocals)
4,The Beatles,Paul McCartney,ba550d0e-adac-4864-b88b-407cab5e76af,bass guitar,1957,1970,Paul McCartney (bass guitar)
5,The Beatles,Ringo Starr,300c4c73-33ac-4255-9d57-4e32627f5e13,drums,1962,1970,Ringo Starr (drums)
6,The Beatles,Stuart Sutcliffe,49a51491-650e-44b3-8085-2f07ac2986dd,bass guitar,1960,1962,Stuart Sutcliffe (bass guitar)
7,The Beatles,John Lennon,4d5447d7-c61c-4120-ba1b-d7f471d385b9,lead vocals,1957,1970,John Lennon (lead vocals)
8,The Beatles,John Lennon,4d5447d7-c61c-4120-ba1b-d7f471d385b9,guitar,1957,1970,John Lennon (guitar)


In [7]:
colors = dict(zip(list(set(df['role'])), ['red', 'blue', 'yellow', 'green']))
print('Correspondance between colors and roles: {}'.format(colors))
df['type'] = df['role'].apply(lambda role: colors[role])
df

Correspondance between colors and roles: {'bass guitar': 'red', 'lead vocals': 'blue', 'drums': 'yellow', 'guitar': 'green'}


,band,member,mbid,role,start,end,label,type
0,The Beatles,George Harrison,42a8f507-8412-4611-854f-926571049fa0,lead vocals,1958,1970,George Harrison (lead vocals),blue
1,The Beatles,George Harrison,42a8f507-8412-4611-854f-926571049fa0,guitar,1958,1970,George Harrison (guitar),green
2,The Beatles,Pete Best,0d4ab0f9-bbda-4ab1-ae2c-f772ffcfbea9,drums,1960,1962,Pete Best (drums),yellow
3,The Beatles,Paul McCartney,ba550d0e-adac-4864-b88b-407cab5e76af,lead vocals,1957,1970,Paul McCartney (lead vocals),blue
4,The Beatles,Paul McCartney,ba550d0e-adac-4864-b88b-407cab5e76af,bass guitar,1957,1970,Paul McCartney (bass guitar),red
5,The Beatles,Ringo Starr,300c4c73-33ac-4255-9d57-4e32627f5e13,drums,1962,1970,Ringo Starr (drums),yellow
6,The Beatles,Stuart Sutcliffe,49a51491-650e-44b3-8085-2f07ac2986dd,bass guitar,1960,1962,Stuart Sutcliffe (bass guitar),red
7,The Beatles,John Lennon,4d5447d7-c61c-4120-ba1b-d7f471d385b9,lead vocals,1957,1970,John Lennon (lead vocals),blue
8,The Beatles,John Lennon,4d5447d7-c61c-4120-ba1b-d7f471d385b9,guitar,1957,1970,John Lennon (guitar),green


We can also add a 'link' columns containing URLs to the MusicBrainz website:

In [8]:
df['link'] = 'https://musicbrainz.org/artist/' + df['mbid'].astype(str)
df

,band,member,mbid,role,start,end,label,type,link
0,The Beatles,George Harrison,42a8f507-8412-4611-854f-926571049fa0,lead vocals,1958,1970,George Harrison (lead vocals),blue,https://musicbrainz.org/artist/42a8f507-8412-4...
1,The Beatles,George Harrison,42a8f507-8412-4611-854f-926571049fa0,guitar,1958,1970,George Harrison (guitar),green,https://musicbrainz.org/artist/42a8f507-8412-4...
2,The Beatles,Pete Best,0d4ab0f9-bbda-4ab1-ae2c-f772ffcfbea9,drums,1960,1962,Pete Best (drums),yellow,https://musicbrainz.org/artist/0d4ab0f9-bbda-4...
3,The Beatles,Paul McCartney,ba550d0e-adac-4864-b88b-407cab5e76af,lead vocals,1957,1970,Paul McCartney (lead vocals),blue,https://musicbrainz.org/artist/ba550d0e-adac-4...
4,The Beatles,Paul McCartney,ba550d0e-adac-4864-b88b-407cab5e76af,bass guitar,1957,1970,Paul McCartney (bass guitar),red,https://musicbrainz.org/artist/ba550d0e-adac-4...
5,The Beatles,Ringo Starr,300c4c73-33ac-4255-9d57-4e32627f5e13,drums,1962,1970,Ringo Starr (drums),yellow,https://musicbrainz.org/artist/300c4c73-33ac-4...
6,The Beatles,Stuart Sutcliffe,49a51491-650e-44b3-8085-2f07ac2986dd,bass guitar,1960,1962,Stuart Sutcliffe (bass guitar),red,https://musicbrainz.org/artist/49a51491-650e-4...
7,The Beatles,John Lennon,4d5447d7-c61c-4120-ba1b-d7f471d385b9,lead vocals,1957,1970,John Lennon (lead vocals),blue,https://musicbrainz.org/artist/4d5447d7-c61c-4...
8,The Beatles,John Lennon,4d5447d7-c61c-4120-ba1b-d7f471d385b9,guitar,1957,1970,John Lennon (guitar),green,https://musicbrainz.org/artist/4d5447d7-c61c-4...


The last preparation step is to transform this Python data structure into a Javascript one that the timesheet library can read. We're going to use the fact that a Python list and a Javascript array are very close.

In [9]:
bubbles = [df.ix[i].to_dict() for i in range(len(df))]
pprint(bubbles)

[{'band': 'The Beatles',
  'end': '1970',
  'label': 'George Harrison (lead vocals)',
  'link': 'https://musicbrainz.org/artist/42a8f507-8412-4611-854f-926571049fa0',
  'mbid': '42a8f507-8412-4611-854f-926571049fa0',
  'member': 'George Harrison',
  'role': 'lead vocals',
  'start': '1958',
  'type': 'blue'},
 {'band': 'The Beatles',
  'end': '1970',
  'label': 'George Harrison (guitar)',
  'link': 'https://musicbrainz.org/artist/42a8f507-8412-4611-854f-926571049fa0',
  'mbid': '42a8f507-8412-4611-854f-926571049fa0',
  'member': 'George Harrison',
  'role': 'guitar',
  'start': '1958',
  'type': 'green'},
 {'band': 'The Beatles',
  'end': '1962',
  'label': 'Pete Best (drums)',
  'link': 'https://musicbrainz.org/artist/0d4ab0f9-bbda-4ab1-ae2c-f772ffcfbea9',
  'mbid': '0d4ab0f9-bbda-4ab1-ae2c-f772ffcfbea9',
  'member': 'Pete Best',
  'role': 'drums',
  'start': '1960',
  'type': 'yellow'},
 {'band': 'The Beatles',
  'end': '1970',
  'label': 'Paul McCartney (lead vocals)',
  'link': 'ht

Perfect. Time to do some javascript. The Jupyter notebook can display javascript plots in an output cell by using the *element.append* magic. I'll display this cell (no. 10) later so that we keep the javascript code (no. 11) above its output, but the *element.append* code must be executed **before** the "new Timesheet" code (so 10 before 11).

Last step: we call the Timesheet javascript command using the CSS/JS libraries loaded above, our input data (referred below as 'bubbles', the cell where we want our graph, and the timeline limit (min and max date). Executing the next cell will fill the output cell just above this block automatically.

In [11]:
from IPython.display import Javascript
Javascript("""
var bubbles = %s;
new Timesheet(bubbles, {
    container: 'timesheet-container',
    type: 'parallel',
    //type: 'serial',
    timesheetYearMin: %s,
    timesheetYearMax: %s,
    theme: 'light'
});
""" % (bubbles, df['start'].min(), df['end'].max()))

<IPython.core.display.Javascript object>

In [10]:
%%javascript
// this must be executed before the "from IPython.display import Javascript" block
element.append('<div id="timesheet-container" style="width: 100%;height: 100%;"></div>');

<IPython.core.display.Javascript object>

We have our timeline now! As you can see the same color is used for the same role consistently. If you can't see the timeline above you can find a copy on [github.io](https://loujine.github.io/musicbrainz-dataviz/).

## Second example

Let's repeat our code on a second band, this time a classical String Quartet group.

In [12]:
band_name = 'Beethoven String Quartet'

In [13]:
def prepare_data(band_name):
    df = sql("""
SELECT b.name AS band,
       m.name AS member,
       m.gid  AS mbid,
       lat.name AS role,
       l.begin_date_year AS start,
       l.end_date_year AS end
FROM artist              AS b
JOIN l_artist_artist     AS laa ON laa.entity1 = b.id
JOIN artist              AS m   ON laa.entity0 = m.id
JOIN link                AS l   ON l.id = laa.link
JOIN link_attribute      AS la  ON la.link = l.id
JOIN link_attribute_type AS lat ON la.attribute_type = lat.id
JOIN link_type           AS lt  ON l.link_type = lt.id
WHERE lt.name = 'member of band'
  AND b.name = %(band_name)s
  AND lat.name != 'original';
""")
    df['start'] = df['start'].fillna(1957).astype(int)
    df['start'] = df['start'].astype(str)
    df['end'] = df['end'].astype(str)
    df['link'] = 'https://musicbrainz.org/artist/' + df['mbid'].astype(str)
    df['label'] = df['member'] + ' (' + df['role'] + ')'
    colors = dict(zip(list(set(df['role'])), ['red', 'blue', 'yellow', 'green']))
    df['type'] = df['role'].apply(lambda role: colors[role])
    df.drop(['band', 'member', 'mbid', 'role'], axis=1, inplace=True)
    return [df.ix[i].to_dict() for i in range(len(df))], df['start'].min(), df['end'].max()

In [14]:
prepare_data(band_name)

([{'end': '1965',
   'label': 'Василий Ширинский (violin)',
   'link': 'https://musicbrainz.org/artist/5d71f6b5-eb33-45b6-807a-fb17545bfb3e',
   'start': '1923',
   'type': 'blue'},
  {'end': '1977',
   'label': 'Дмитрий Цыганов (violin)',
   'link': 'https://musicbrainz.org/artist/d0aeb2ad-308e-41ce-b767-8e7227284a91',
   'start': '1923',
   'type': 'blue'},
  {'end': '1990',
   'label': 'Nikolai Zabavnikov (violin)',
   'link': 'https://musicbrainz.org/artist/19c80b9e-5b90-4dde-80e6-9e906862aa22',
   'start': '1965',
   'type': 'blue'},
  {'end': '1990',
   'label': 'Олег Крыса (violin)',
   'link': 'https://musicbrainz.org/artist/bfe32af2-0725-4d2e-97a8-36894b1404a1',
   'start': '1977',
   'type': 'blue'},
  {'end': '1974',
   'label': 'Сергей Петрович Ширинский (cello)',
   'link': 'https://musicbrainz.org/artist/de91fd01-2067-436f-9dcb-4ee07a89b0df',
   'start': '1923',
   'type': 'yellow'},
  {'end': '1964',
   'label': 'Вадим Васильевич Борисовский (viola)',
   'link': 'https:/

In [16]:
from IPython.display import Javascript
Javascript("""
var bubbles = %s;
new Timesheet(bubbles, {
    container: 'timesheet-container2',
    type: 'parallel',
    timesheetYearMin: %s,
    timesheetYearMax: %s,
    theme: 'light'
});
""" % prepare_data(band_name))

<IPython.core.display.Javascript object>

In [15]:
%%javascript
// this must be executed before the "from IPython.display import Javascript" block
element.append('<div id="timesheet-container2" style="width: 100%;height: 100%;"></div>');

<IPython.core.display.Javascript object>

The timeline is probably larger than your window, use the slider to see when members changed in the String Quartet group.

One possible improvement would be to fetch the instrument credit ('1st violin', '2nd violin'...)  to have more precise roles.

## Conclusion

With this notebook I hoped I showed that testing a new plotting library to display data from MusicBrainz is not very complicated